<a href="https://colab.research.google.com/github/swapnilbokde/BCJ-Shear-Strength-Prediction-Model-XGBoost-/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install plotly
!pip install scikit-learn==1.0.2
!pip install plotly==5.4.0
# !pip install flaml
!pip install FLAML==0.7.1
!pip install ppscore

# !pip install -U scikit-learn

     |████████████████████████████████| 24.8 MB 5.4 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ppscore 1.2.0 requires scikit-learn<1.0.0,>=0.20.2, but you have scikit-learn 1.0.2 which is incompatible.


  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [ ]:
import pandas as pd  # Import Panda Library
import numpy as np # Import Numpy Library

import plotly.io as pio
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.figure_factory as ff
pd.options.plotting.backend = "plotly"
pio.templates.default = 'simple_white'

# from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pickle
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

from flaml import AutoML

import xgboost as xgb

import ppscore as pps

import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_df = pd.read_excel('/content/drive/My Drive/data/Beam_Column_Joint.xlsx') # Assigning data_df variable of our excel sheet to panda  data framework

In [ ]:
data_df.drop(0, inplace=True) # Removing the serial number column from our data set

In [ ]:
data_df.drop('SI. No.', axis=1, inplace=True)  # Removing the serial number column from our data set

In [ ]:
data_df.columns = ['b_j', 'b_b_by_b_c', 'f_c', 'T_Ast', 'M_R', 'L', 'f_yb', 'phi_s', 'P_y', 'x_b', 'h_b_by_h_c', 'tau'] # Labelling input and output variables in column vector.

In [ ]:
data_df.info() # Displaying column vector of input and output variables

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 1 to 136
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   b_j         136 non-null    object 
 1   b_b_by_b_c  136 non-null    object 
 2   f_c         136 non-null    object 
 3   T_Ast       136 non-null    object 
 4   M_R         136 non-null    object 
 5   L           136 non-null    float64
 6   f_yb        136 non-null    object 
 7   phi_s       136 non-null    object 
 8   P_y         136 non-null    object 
 9   x_b         136 non-null    object 
 10  h_b_by_h_c  136 non-null    object 
 11  tau         136 non-null    object 
dtypes: float64(1), object(11)
memory usage: 13.8+ KB


In [ ]:
data_df # Displaying original data in pandas data frame

,b_j,b_b_by_b_c,f_c,T_Ast,M_R,L,f_yb,phi_s,P_y,x_b,h_b_by_h_c,tau
1,280,0.8667,30.88,0.86,1.7959,1500.0,520,0,0.15,0.1538,1.5,3.756
2,280,0.8667,30.88,0.86,1.7959,1500.0,520,0,0.15,0.1538,1.5,2.6095
3,280,0.8667,42.08,0.86,1.7959,1500.0,520,0,0.15,0.1538,1.5,4.7464
4,280,0.8667,31.04,0.86,1.7959,1500.0,520,0,0.15,0.1538,1.5,4.0619
5,280,0.8667,31.6,0.86,1.7959,1500.0,520,0,0.15,0.1538,1.5,4.106
...,...,...,...,...,...,...,...,...,...,...,...,...
132,190,0.7273,41,1.59,2.2051,890.0,391,0.3567,0.0934,0.325,1,5.0263
133,190,0.7273,39.7,1.59,2.3077,890.0,391,0.4769,0.0964,0.325,1,5.0096
134,190,0.7273,37.4,1.59,3.2308,890.0,391,1.6762,0,0.325,1,4.9402
135,190,0.7273,39.7,1.59,1.0256,890.0,391,0.9799,0,0.325,1,5.1938


In [ ]:
data_df['L'] = data_df['L'].astype(int)

In [ ]:
data_df['tau'] = data_df['tau'].astype(float)

In [ ]:
pps_matrix = pps.matrix(data_df) # Applying pps score to pandas data frame, not exactly, but slighly similar to correlation coefficient

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:668: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=4.

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:668: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=4.

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:668: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=4.

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:668: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=4.

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:668: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=4.

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:668: UserWarning:

The least populated class in y ha

In [ ]:
pps_matrix

,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
0,b_j,b_j,1.000000,predict_itself,True,None,0.000000,1.000000,None
1,b_j,b_b_by_b_c,0.971262,classification,True,weighted F1,0.242647,0.978235,DecisionTreeClassifier()
2,b_j,f_c,0.115707,classification,True,weighted F1,0.036765,0.148218,DecisionTreeClassifier()
3,b_j,T_Ast,0.427675,classification,True,weighted F1,0.058824,0.461341,DecisionTreeClassifier()
4,b_j,M_R,0.205949,classification,True,weighted F1,0.044118,0.240981,DecisionTreeClassifier()
...,...,...,...,...,...,...,...,...,...
139,tau,phi_s,0.074400,classification,True,weighted F1,0.057088,0.127241,DecisionTreeClassifier()
140,tau,P_y,0.163900,classification,True,weighted F1,0.088235,0.237674,DecisionTreeClassifier()
141,tau,x_b,0.394148,classification,True,weighted F1,0.080882,0.443151,DecisionTreeClassifier()
142,tau,h_b_by_h_c,0.260551,classification,True,weighted F1,0.257353,0.450851,DecisionTreeClassifier()


In [ ]:
pps_score_df = pps_matrix[(pps_matrix['y']=='tau') & (pps_matrix['x']!='tau')] # Calculating pps matrix for x to y with MAE baseline score obtained from navie model and MAE orignal score obtained from Decision Tree Regressor Model
pps_score_df

,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
11,b_j,tau,0.309885,regression,True,mean absolute error,1.320352,0.911195,DecisionTreeRegressor()
23,b_b_by_b_c,tau,0.324924,regression,True,mean absolute error,1.320352,0.891338,DecisionTreeRegressor()
35,f_c,tau,0.375485,regression,True,mean absolute error,1.320352,0.824580,DecisionTreeRegressor()
47,T_Ast,tau,0.597157,regression,True,mean absolute error,1.320352,0.531894,DecisionTreeRegressor()
59,M_R,tau,0.264256,regression,True,mean absolute error,1.320352,0.971441,DecisionTreeRegressor()
71,L,tau,0.313192,regression,True,mean absolute error,1.320352,0.906828,DecisionTreeRegressor()
83,f_yb,tau,0.397803,regression,True,mean absolute error,1.320352,0.795112,DecisionTreeRegressor()
95,phi_s,tau,0.094937,regression,True,mean absolute error,1.320352,1.195002,DecisionTreeRegressor()
107,P_y,tau,0.042687,regression,True,mean absolute error,1.320352,1.263991,DecisionTreeRegressor()
119,x_b,tau,0.439079,regression,True,mean absolute error,1.320352,0.740613,DecisionTreeRegressor()


In [ ]:
pps_score_df[['x', 'ppscore']].sort_values('ppscore').plot(kind='bar', x='ppscore', y='x',) # Plotting pps score on x axis and input features on y axis, thus displaying the importance of various features

In [ ]:
# X = data_df.drop('tau',axis=1)
# y = data_df['tau']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
feature_cols = data_df.columns.to_list() # Assigning variable feature_cols to data_df for listing the variables
feature_cols.remove('tau') # Defining feature_cols from data_df by removing tau as an input vector
feature_cols # Displaying defined variable feature_cols

['b_j',
 'b_b_by_b_c',
 'f_c',
 'T_Ast',
 'M_R',
 'L',
 'f_yb',
 'phi_s',
 'P_y',
 'x_b',
 'h_b_by_h_c']

In [ ]:
x_mm_sc = MinMaxScaler() # Defining Normalization
y_mm_sc = MinMaxScaler()

In [ ]:
X=x_mm_sc.fit_transform(data_df[feature_cols]) # normalizing and transforming into scaled feature variables
y=y_mm_sc.fit_transform(data_df[['tau']]) # normalizing and transforming into scaled output variable tau

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Splitting the entire data into 80% as training set and 20% as testing set and randomlying shulling the data by defining random state of 42

In [ ]:
X.shape, y.shape # Recalling the size of input and output varaibles

((136, 11), (136, 1))

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape # Recalling the size of train and test set of input and output

((108, 11), (28, 11), (108, 1), (28, 1))

In [ ]:
y.shape

(136, 1)

In [ ]:
data_df['tau'].hist()

In [ ]:
pd.Series(y.reshape(136,)).hist() # Restructuing the output in panda data frame to a single column vector

In [ ]:
automl = AutoML() # Assigning the variable automl and calling the fuction AutoML, the algorithms used are lgbm, random forrest, xtremly boosted regression trees, extra tre regressor and xgb limit depth
automl.fit(X_train, y_train, task="regression", metric='mse', time_budget=240, estimator_list=['xgboost']) # AutoML function applied to training set, defining task of regression, using evaluation metrics as MSE, and duration of training is defined as 2 hours

Streaming output truncated to the last 5000 lines.
[flaml.automl: 03-13 08:34:57] {2029} INFO -  at 30.3s,	estimator xgboost's best error=0.0073,	best estimator xgboost's best error=0.0073
[flaml.automl: 03-13 08:34:57] {1826} INFO - iteration 458, current learner xgboost
[flaml.automl: 03-13 08:34:57] {2029} INFO -  at 30.4s,	estimator xgboost's best error=0.0073,	best estimator xgboost's best error=0.0073
[flaml.automl: 03-13 08:34:57] {1826} INFO - iteration 459, current learner xgboost
[flaml.automl: 03-13 08:34:57] {2029} INFO -  at 30.4s,	estimator xgboost's best error=0.0073,	best estimator xgboost's best error=0.0073
[flaml.automl: 03-13 08:34:57] {1826} INFO - iteration 460, current learner xgboost
[flaml.automl: 03-13 08:34:57] {2029} INFO -  at 30.5s,	estimator xgboost's best error=0.0073,	best estimator xgboost's best error=0.0073
[flaml.automl: 03-13 08:34:57] {1826} INFO - iteration 461, current learner xgboost
[flaml.automl: 03-13 08:34:57] {2029} INFO -  at 30.6s,	estim

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 101, 'max_leaves': 5, 'min_child_weight': 0.13480517549507565, 'learning_rate': 0.32027197557559395, 'subsample': 0.6825667215588685, 'colsample_bylevel': 0.6826272512831161, 'colsample_bytree': 0.8968471629058937, 'reg_alpha': 0.11025199954608665, 'reg_lambda': 0.07346503490819731}
Best accuracy on validation data: 0.9936
Training duration of best run: 0.02221 s


In [ ]:
xgb_model = xgb.XGBRegressor(base_score=0.5, booster='gbtree',
             colsample_bylevel=0.6826272512831161, colsample_bynode=1,
             colsample_bytree=0.8968471629058937, gamma=0, gpu_id=0,
            #  gpu_id=-1
             importance_type='gain', interaction_constraints='',
             learning_rate=0.32027197557559395, max_delta_step=0, max_depth=5,max_leaves=5, 
             min_child_weight=0.13480517549507565, missing=np.nan,
            #  monotone_constraints='()',
             n_estimators=101, n_jobs=-1,
             num_parallel_tree=1, random_state=0,
             reg_alpha=0.11025199954608665, reg_lambda=0.07346503490819731,
             scale_pos_weight=1, subsample=0.6825667215588685,
             tree_method='exact', use_label_encoder=False,
             validate_parameters=1, verbosity=0)

In [ ]:
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# CV model
cv = KFold(n_splits=10, random_state=1, shuffle=True)

# model = xgboost.XGB()
# evaluate model
scores_train = cross_val_score(xgb_model, X_train, y_train, scoring='r2', cv=cv, n_jobs=-1)
#scores_test = cross_val_score(xgb_model, X_test, y_test, scoring='r2', cv=cv, n_jobs=-1)
# report performance
scores_train
#scores_test
# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# results = cross_val_score(xgb_model, X_train, y_train, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

array([0.19139343, 0.91871442, 0.90641546, 0.96055404, 0.88533202,
       0.80377266, 0.87250065, 0.40650756, 0.92889045, 0.85606243])

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

# model = xgboost.XGB()
# evaluate model
scores_test = cross_val_score(xgb_model, X_test, y_test, scoring='r2', cv=cv, n_jobs=-1)
#scores_test = cross_val_score(xgb_model, X_test, y_test, scoring='r2', cv=cv, n_jobs=-1)
# report performance
scores_test

array([  0.96901425,   0.75513278,   0.92223192,  -0.57581614,
         0.61207329, -13.55397563,   0.51454413,  -0.7552748 ,
         0.92073611,   0.93019713])

In [ ]:
xgb_model.fit(X_train, y_train) # got latest model as xgb with the following hyperparameters and model is trained again on the training set

XGBRegressor(colsample_bylevel=0.6826272512831161,
             colsample_bytree=0.8968471629058937, gpu_id=0,
             interaction_constraints='', learning_rate=0.32027197557559395,
             max_depth=5, max_leaves=5, min_child_weight=0.13480517549507565,
             n_estimators=101, n_jobs=-1, num_parallel_tree=1,
             reg_alpha=0.11025199954608665, reg_lambda=0.07346503490819731,
             subsample=0.6825667215588685, tree_method='exact',
             use_label_encoder=False, validate_parameters=1, verbosity=0)

In [ ]:
xgb_model.score(X_train, y_train) # Evaluating the R2 score on x train data and y train data

0.9943006827909154

In [ ]:
xgb_model.score(X_test, y_test) # Evaluating the R2 score on x test data and y test data

0.9724623059873754

In [ ]:
y_test_predicted = xgb_model.predict(X_test) # Assinging variable y_test_predicted to the output of the test data from xgb model

In [ ]:
from sklearn.metrics import mean_squared_error # importing MSE function from sklearn library 
mean_squared_error(y_test, y_test_predicted) # MSE for scaled test data for Xgboost model

0.0010692722562864404

In [ ]:
y_test_predicted.shape

(28,)

In [ ]:
y_test.shape

(28, 1)

In [ ]:
actual_test_tau = y_mm_sc.inverse_transform(y_test) # Transforming scaled test data to original fomat
actual_test_tau

array([[2.3171],
       [4.4383],
       [3.4694],
       [2.255 ],
       [5.0096],
       [6.8533],
       [4.1   ],
       [4.752 ],
       [4.7217],
       [7.2844],
       [4.6333],
       [3.3798],
       [4.025 ],
       [2.0943],
       [6.015 ],
       [2.2925],
       [5.0096],
       [6.4303],
       [4.106 ],
       [6.2264],
       [5.055 ],
       [4.9928],
       [3.95  ],
       [4.4517],
       [4.4383],
       [4.0762],
       [6.686 ],
       [6.4993]])

In [ ]:
predicted_test_tau = y_mm_sc.inverse_transform(y_test_predicted.reshape(28,1)) # Transforming scaled test data to original fomat
predicted_test_tau

array([[2.641944 ],
       [4.4265046],
       [2.991821 ],
       [2.5378172],
       [4.8878217],
       [7.0657463],
       [3.980801 ],
       [5.2740207],
       [4.6208134],
       [6.933584 ],
       [4.7448387],
       [3.7211254],
       [4.053218 ],
       [2.2320392],
       [6.022106 ],
       [2.358189 ],
       [4.9864144],
       [6.3309646],
       [3.884539 ],
       [5.919602 ],
       [5.08371  ],
       [5.0655766],
       [3.9934266],
       [4.45901  ],
       [4.6069922],
       [3.5970023],
       [6.933718 ],
       [6.563622 ]], dtype=float32)

In [ ]:
mean_squared_error(actual_test_tau, predicted_test_tau) # MSE of non-scaled test data for Xgboost Model

0.05471283150108116

In [ ]:
math.sqrt(mean_squared_error(actual_test_tau, predicted_test_tau)) # RMSE of non-scaled test data for XGBoost Model

0.233907741430422

In [ ]:
from sklearn.metrics import mean_absolute_error # importing MAE function from sklearn library 
mae_test= mean_absolute_error(actual_test_tau, predicted_test_tau)
mae_test

0.17781187105178842

In [ ]:
from sklearn.metrics import r2_score # importing R2 function from sklearn library
r2_score_test=r2_score(actual_test_tau, predicted_test_tau)
r2_score_test

0.9724622960563558

In [ ]:
adjusted_r2_score_test= 1 - (1-r2_score_test)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1) # adjusted R square
adjusted_r2_score_test

0.9535301245951004

In [ ]:
y_train_predicted = xgb_model.predict(X_train)

In [ ]:
actual_train_tau = y_mm_sc.inverse_transform(y_train) # Transforming actual scaled train data to original fomat
actual_train_tau

array([[4.31  ],
       [6.7071],
       [5.0885],
       [2.9455],
       [6.3229],
       [6.9321],
       [3.756 ],
       [3.63  ],
       [6.4371],
       [4.025 ],
       [5.706 ],
       [4.945 ],
       [2.6409],
       [4.4383],
       [5.7927],
       [2.2925],
       [4.5183],
       [7.1913],
       [4.9689],
       [7.4441],
       [4.4   ],
       [6.105 ],
       [3.1418],
       [6.4069],
       [4.2869],
       [6.085 ],
       [5.0263],
       [3.95  ],
       [4.6342],
       [3.985 ],
       [5.085 ],
       [6.9498],
       [6.0733],
       [4.7964],
       [3.756 ],
       [6.34  ],
       [4.9474],
       [8.3477],
       [6.0119],
       [2.143 ],
       [6.05  ],
       [8.5683],
       [5.706 ],
       [4.7283],
       [2.9   ],
       [3.5935],
       [6.7958],
       [5.0431],
       [3.3798],
       [5.8   ],
       [8.6303],
       [4.0619],
       [4.4383],
       [2.3865],
       [3.075 ],
       [4.6086],
       [4.6345],
       [6.429 ],
       [4.9522

In [ ]:
predicted_train_tau = y_mm_sc.inverse_transform(y_train_predicted.reshape(108,1)) # Transforming predicted scaled train data to original fomat
predicted_train_tau

array([[4.2988057],
       [6.5991826],
       [5.0676193],
       [2.9448402],
       [6.1392903],
       [6.973845 ],
       [3.3206985],
       [3.908659 ],
       [6.3309646],
       [4.041344 ],
       [5.703336 ],
       [4.9840302],
       [2.6566288],
       [4.433882 ],
       [5.7415905],
       [2.238357 ],
       [4.469663 ],
       [6.9864454],
       [4.994994 ],
       [7.4714766],
       [4.4473324],
       [6.075653 ],
       [3.083808 ],
       [6.5355935],
       [4.3346534],
       [6.089083 ],
       [5.0003214],
       [4.075074 ],
       [4.64664  ],
       [4.0495653],
       [5.031838 ],
       [6.938727 ],
       [5.976044 ],
       [4.74362  ],
       [3.6008792],
       [6.3805633],
       [4.939797 ],
       [8.343124 ],
       [6.012193 ],
       [2.2320392],
       [6.043619 ],
       [8.300111 ],
       [5.780135 ],
       [4.728474 ],
       [2.9235024],
       [3.5963795],
       [6.794127 ],
       [5.0098767],
       [3.363908 ],
       [5.7081594],


In [ ]:
mean_squared_error(actual_train_tau, predicted_train_tau) # MSE of non-scaled train data

0.016442355792109977

In [ ]:
math.sqrt(mean_squared_error(actual_train_tau, predicted_train_tau))  # RMSE of non-scaled train data

0.12822774969603878

In [ ]:
mae_train= mean_absolute_error(actual_train_tau, predicted_train_tau)
mae_train

0.07832951343324446

In [ ]:
r2_score_train=r2_score(actual_train_tau, predicted_train_tau)
r2_score_train

0.9943006838557897

In [ ]:
adjusted_r2_score_train= 1 - (1-r2_score_train)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1) # adjusted R square
adjusted_r2_score_train

0.9936476372142656

In [ ]:
x_train_df = pd.DataFrame(x_mm_sc.inverse_transform(X_train), columns = feature_cols)
x_train_df.head()
x_train_df

,b_j,b_b_by_b_c,f_c,T_Ast,M_R,L,f_yb,phi_s,P_y,x_b,h_b_by_h_c
0,200.00,1.0000,35.00,0.5818,4.3100,900.0,510.0,1.6289,0.1429,0.1333,1.5000
1,385.00,0.7111,76.60,1.5958,3.1000,1900.0,491.0,0.7346,0.0148,0.3125,1.0000
2,190.00,0.7273,35.10,1.5900,1.8461,890.0,391.0,0.7578,0.0000,0.3250,1.0000
3,275.00,0.8333,39.20,0.8825,3.2565,1000.0,570.0,0.0000,0.0000,0.1200,1.6667
4,275.00,0.8333,43.20,0.8825,1.6600,1000.0,515.0,0.8009,0.1684,0.1536,1.6667
...,...,...,...,...,...,...,...,...,...,...,...
103,200.00,1.0000,18.96,0.8713,2.8750,1000.0,585.0,0.0000,0.0607,0.1400,1.5000
104,381.00,0.8750,27.60,1.4100,2.1576,2133.0,482.7,1.8563,0.0000,0.2500,1.1250
105,200.00,1.0000,22.00,0.8552,5.5600,900.0,495.0,1.4633,0.2273,0.1400,1.5000
106,336.55,0.8929,73.80,1.9272,1.4000,1600.0,437.0,1.4328,0.0685,0.2557,1.4286


In [ ]:
len(x_train_df)

108

In [ ]:
y_train_df = pd.DataFrame(y_mm_sc.inverse_transform(y_train), columns = ['tau'])
y_train_df.head()

,tau
0,4.3100
1,6.7071
2,5.0885
3,2.9455
4,6.3229


In [ ]:
len(y_train_df)

108

In [ ]:
train_df = x_train_df.join(y_train_df)

In [ ]:
predicted_train_df = pd.DataFrame(predicted_train_tau, columns=['predicted_tau'])
predicted_train_df.head()

,predicted_tau
0,4.298806
1,6.599183
2,5.067619
3,2.944840
4,6.139290


In [ ]:
len(predicted_train_df)

108

In [ ]:
train_df = train_df.join(predicted_train_df)

In [ ]:
x_test_df = pd.DataFrame(x_mm_sc.inverse_transform(X_test), columns = feature_cols)
x_test_df.head()

,b_j,b_b_by_b_c,f_c,T_Ast,M_R,L,f_yb,phi_s,P_y,x_b,h_b_by_h_c
0,275.0,0.8333,39.2,0.8825,1.6283,1000.0,570.00,0.0000,0.0000,0.1200,1.6667
1,200.0,1.0000,31.6,0.8378,1.7600,1000.0,580.00,0.2701,0.0500,0.2400,1.0000
2,350.0,1.0000,19.6,0.7860,1.4000,1780.0,416.75,1.1083,0.0000,0.1800,1.1429
3,200.0,1.0000,31.6,0.2909,1.6600,1000.0,580.00,0.7780,0.0500,0.0667,1.5000
4,190.0,0.7273,41.9,1.5900,2.6154,890.0,391.00,0.6935,0.0914,0.3250,1.0000


In [ ]:
len(x_test_df)

28

In [ ]:
y_test_df = pd.DataFrame(y_mm_sc.inverse_transform(y_test), columns = ['tau'])
y_test_df.head()

,tau
0,2.3171
1,4.4383
2,3.4694
3,2.2550
4,5.0096


In [ ]:
len(y_test_df)

28

In [ ]:
test_df = x_test_df.join(y_test_df)

In [ ]:
predicted_test_df = pd.DataFrame(predicted_test_tau, columns=['predicted_tau'])
predicted_test_df.head()

,predicted_tau
0,2.641944
1,4.426505
2,2.991821
3,2.537817
4,4.887822


In [ ]:
test_df = test_df.join(predicted_test_df)

In [ ]:
# train_df.round(4, inplace=True)
new_data=train_df.round(4)
new_data

,b_j,b_b_by_b_c,f_c,T_Ast,M_R,L,f_yb,phi_s,P_y,x_b,h_b_by_h_c,tau,predicted_tau
0,200.00,1.0000,35.00,0.5818,4.3100,900.0,510.0,1.6289,0.1429,0.1333,1.5000,4.3100,4.2988
1,385.00,0.7111,76.60,1.5958,3.1000,1900.0,491.0,0.7346,0.0148,0.3125,1.0000,6.7071,6.5992
2,190.00,0.7273,35.10,1.5900,1.8461,890.0,391.0,0.7578,0.0000,0.3250,1.0000,5.0885,5.0676
3,275.00,0.8333,39.20,0.8825,3.2565,1000.0,570.0,0.0000,0.0000,0.1200,1.6667,2.9455,2.9448
4,275.00,0.8333,43.20,0.8825,1.6600,1000.0,515.0,0.8009,0.1684,0.1536,1.6667,6.3229,6.1393
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,200.00,1.0000,18.96,0.8713,2.8750,1000.0,585.0,0.0000,0.0607,0.1400,1.5000,2.8750,2.8515
104,381.00,0.8750,27.60,1.4100,2.1576,2133.0,482.7,1.8563,0.0000,0.2500,1.1250,7.3612,7.2509
105,200.00,1.0000,22.00,0.8552,5.5600,900.0,495.0,1.4633,0.2273,0.1400,1.5000,5.5600,5.6395
106,336.55,0.8929,73.80,1.9272,1.4000,1600.0,437.0,1.4328,0.0685,0.2557,1.4286,8.5336,8.5613


In [ ]:
new_data.to_csv('/content/drive/My Drive/data/BCJ_training_data1.csv', index=False)

In [ ]:
test_df.to_csv('/content/drive/My Drive/data/BCJ_test_data1.csv', index=False)

In [ ]:
actual_overall_tau=np.concatenate((actual_train_tau,actual_test_tau),axis=0)
actual_overall_tau

array([[4.31  ],
       [6.7071],
       [5.0885],
       [2.9455],
       [6.3229],
       [6.9321],
       [3.756 ],
       [3.63  ],
       [6.4371],
       [4.025 ],
       [5.706 ],
       [4.945 ],
       [2.6409],
       [4.4383],
       [5.7927],
       [2.2925],
       [4.5183],
       [7.1913],
       [4.9689],
       [7.4441],
       [4.4   ],
       [6.105 ],
       [3.1418],
       [6.4069],
       [4.2869],
       [6.085 ],
       [5.0263],
       [3.95  ],
       [4.6342],
       [3.985 ],
       [5.085 ],
       [6.9498],
       [6.0733],
       [4.7964],
       [3.756 ],
       [6.34  ],
       [4.9474],
       [8.3477],
       [6.0119],
       [2.143 ],
       [6.05  ],
       [8.5683],
       [5.706 ],
       [4.7283],
       [2.9   ],
       [3.5935],
       [6.7958],
       [5.0431],
       [3.3798],
       [5.8   ],
       [8.6303],
       [4.0619],
       [4.4383],
       [2.3865],
       [3.075 ],
       [4.6086],
       [4.6345],
       [6.429 ],
       [4.9522

In [ ]:
len(actual_overall_tau)

136

In [ ]:
predicted_overall_tau=np.concatenate((predicted_train_tau,predicted_test_tau),axis=0)
predicted_overall_tau

array([[4.2988057],
       [6.5991826],
       [5.0676193],
       [2.9448402],
       [6.1392903],
       [6.973845 ],
       [3.3206985],
       [3.908659 ],
       [6.3309646],
       [4.041344 ],
       [5.703336 ],
       [4.9840302],
       [2.6566288],
       [4.433882 ],
       [5.7415905],
       [2.238357 ],
       [4.469663 ],
       [6.9864454],
       [4.994994 ],
       [7.4714766],
       [4.4473324],
       [6.075653 ],
       [3.083808 ],
       [6.5355935],
       [4.3346534],
       [6.089083 ],
       [5.0003214],
       [4.075074 ],
       [4.64664  ],
       [4.0495653],
       [5.031838 ],
       [6.938727 ],
       [5.976044 ],
       [4.74362  ],
       [3.6008792],
       [6.3805633],
       [4.939797 ],
       [8.343124 ],
       [6.012193 ],
       [2.2320392],
       [6.043619 ],
       [8.300111 ],
       [5.780135 ],
       [4.728474 ],
       [2.9235024],
       [3.5963795],
       [6.794127 ],
       [5.0098767],
       [3.363908 ],
       [5.7081594],


In [ ]:
len(predicted_overall_tau)

136

In [ ]:
mean_squared_error(actual_overall_tau, predicted_overall_tau) # MSE of non-scaled train data

0.0243215713792511

In [ ]:
math.sqrt(mean_squared_error(actual_overall_tau, predicted_overall_tau))  # RMSE of non-scaled train data

0.1559537475639848

In [ ]:
mae= mean_absolute_error(actual_overall_tau, predicted_overall_tau)
mae

0.09881117529588587

In [ ]:
r2_score=r2_score(actual_overall_tau, predicted_overall_tau)
r2_score

0.9911273229474817

In [ ]:
adjusted_r2_score= 1 - (1-r2_score)*(len(predicted_overall_tau)-1)/(len(predicted_overall_tau)-11-1) # adjusted R square
adjusted_r2_score

0.9903402306283068